# 1-D Parabolic model problem (Unit 3.1)

We are solving the unsteady heat equation

$$\text{find } u:[0,T]\rightarrow H_{0,D}^1 \hspace{1em} \int_\Omega \partial_t u v + \int_\Omega \epsilon \nabla u \nabla v + b \cdot \nabla u v = \int f v $$ 

$$\forall v \in H_{0,D}^1 \hspace{2em} u(t=0) = u_0 $$

with a suitable advective field b
(the wind).

In [1]:
import netgen.gui
%gui tk
import tkinter
from math import pi
from ngsolve import *
import ngsolve.internal as ngint
ngint.viewoptions.drawedges = 1

from mesh_util import uniform_1d_mesh

We generate geometry $(−1,1)$, mesh (Dirichlet boundaries everywhere):

In [2]:

mesh = Mesh( uniform_1d_mesh(-1,1,100))
fes = H1(mesh, order=3, dirichlet="right|left")
u = fes.TrialFunction()
v = fes.TestFunction()

time = 0.0
dt = 0.001

We define the field $b$ (the wind) as $b(x)=2x(1−x^2)$.

In [3]:
b = CoefficientFunction(2*x*(1-x*x))
Draw(b,mesh,"wind")


We define and assemble bilinear forms for convection-diffusion stiffness and mass matrix separately.

Note that we choose a non-symmetric memory layout for the mass matrix so that a and m have the same sparsity pattern.

In [4]:
a = BilinearForm(fes, symmetric=False)
a += SymbolicBFI (0.01*grad(u)*grad(v) + b*grad(u)*v)
a.Assemble()

m = BilinearForm(fes, symmetric=False)
m += SymbolicBFI (u*v)
m.Assemble()


We want to use a simple implicit Euler time stepping method, i.e.
$$(M+\Delta t\ A)u^{n+1}=M u^n+f^{n+1}$$

or in an incremental form:
$$\underbrace{(M+\Delta t\ A)}_{M^*}(u^{n+1}−u^n)=−\Delta t\ A u^n+f^{n+1}.$$

The incremental form has the advantage that $u^{n+1}−u^n$ has homogeneous boundary conditions (unless boundary conditions are time-dependent).

For the time stepping method we want to set up linear combinations of matrices.

(Only) if the sparsity pattern of the matrices agree we can copy the pattern and sum up the entries.

First, we create a matrix of the same size and sparsity pattern as m.mat:

In [5]:
mstar = m.mat.CreateMatrix()
# print(mstar)

To access the entries we use the vector of nonzero-entries:

In [6]:
# print(mstar.AsVector())

In [7]:
mstar.AsVector().data = m.mat.AsVector() + dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())

### Note: 
The above technique is used below - just not right away...

We set the r.h.s. $f=exp(−6(x+5)^2)−exp(−6(x−5)^2)$

In [8]:
f = LinearForm(fes)
gaussp = exp(-6*(x+0.5)*(x+0.5))-exp(-6*(x-0.5)*(x-0.5))
Draw(gaussp,mesh,"f")
f += SymbolicLFI(gaussp*v)
f.Assemble()

and the initial data: $u_0=(1−y2)x$

In [9]:
gfu = GridFunction(fes)
gfu.Set(1-x*x)
Draw(gfu,mesh,"u")

In [11]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 10.671999999999898

 11.52699999999984

 14.738999999999814

 17.937000000000882

 18.723000000000503

 19.566000000000034

 19.999999999999794


## Supplementary 1: time-dependent r.h.s. data

Next, we want to consider time-dependent r.h.s. data $f=f(t)$:

To this end, we introduce a parameter t representing the time.

A Parameter is a constant CoefficientFunction the value of which can be changed with the Set-function.

In [12]:
t = Parameter(0.0)

An example of a time-dependent coefficient that we want to use as r.h.s. in the following is

In [15]:
omega=1
tt = 0
gausspt = exp(-6*(x+sin(omega*t))*(x+sin(omega*t)))-exp(-6*(x-sin(omega*t))*(x-sin(omega*t)))
Draw(gausspt,mesh,"ft")
time = 0.0
while time < 10 - 0.5 * dt:
    t.Set(time)
    tt = time
    Redraw(blocking=True)
    time += 1e-5



Accordingly we define a different linear form which then has to be assembled in every time step.

In [16]:
ft = LinearForm(fes)
ft += SymbolicLFI(gausspt*v)
time = 0.0
t.Set(0.0)
gfu.Set(1-x*x)
Draw(gfu,mesh,"u")

In [17]:
tstep = 20 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    ft.Assemble()
    res.data = dt * ft.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 0.6470000000000005

 1.260999999999972

 1.8529999999999067

 2.4639999999998397

 4.209999999999741

...

 17.37199999999825

 18.571999999999715

 19.225000000000513

 19.848000000001274

 20.00000000000146


## Supplementary 2: Time dependent boundary conditions

Now, we want to consider changing boundary conditions $u|_{\partial \Omega} = u_D(t)$ and set $f=0$.

We use the simple implicit Euler time stepping method again for this problem. Now, we directly use the non-incremental form:
$$(M+\Delta t\ A)u^{n+1}=M u^n$$
We homogenize w.r.t. to the boundary conditions, i.e. we split $u^{n+1}=u_0^{n+1}+u_D^{n+1}$ where $u_D^{n+1}$ is some function in the FESpace with the correct boundary condition and $u_0^{n+1}$ is the sought for function with zero boundary values:
$$\underbrace{(M+\Delta t\ A)}_{M^*} u_0^{n+1} = Mu^n - \underbrace{(M+\Delta t\ A)}_{M^*} u_D^{n+1}$$

In [18]:
uD = CoefficientFunction( [(1-x*x)*IfPos(sin(0.3*pi*t),sin(0.3*pi*t),0),0,0,0])
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
visoptions.mminval = 0.0
visoptions.mmaxval = 0.9
visoptions.deformation = 0
visoptions.autoscale = 0


In [19]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
while t_intermediate < tstep - 0.5 * dt:
    t.Set(time+t_intermediate+dt)
    #non-incremental form 
    res.data = m.mat * gfu.vec
    gfu.Set(uD,BND)
    res.data -= mstar * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
print("")
time+=t_intermediate

 0.7530000000000006

 1.432999999999953

 2.1409999999998752

 3.594999999999715

 4.348999999999787

 5.741000000000252

 6.465000000000494

 7.214000000000744

 9.47100000000019

 9.999999999999897


## Supplementary 3: Singly diagonally implicit Runge-Kutta methods

We consider more sophisticated time integration methods, SDIRK methods. To simplify presentation we set $f=0$.

SDIRK methods for the semi-discrete problem $\cfrac{d}{dt}u=M^{−1}F(u)=−M^{−1}\cdot A u$ are of the form:
$u^{n+1}=u_n+\Delta t\ M^{−1} \sum_{i=0}^{s−1} b_i k_i$ with $k_i=−A u_i$ where $u_i$ is the solution to 
$$(M+a^∗\Delta t A) u_i=M u^n−\Delta t \sum_{j=0}^{i−1} a_{ij}k_j, \hspace{2em} i=0,\dots,s−1.$$
The coefficients $a$,$b$ and $c$ are stored in the butcher tableau: 


 <table style="width:50%; border-collapse:collapse;">
  <tr>
    <td style='border-right: 1px solid black;'>$c_0$</th>
    <td>$a_{00}$</td>
    <td>0</td>
    <td>$\ddots$</td>
    <td>&nbsp;</td>
  </tr>
  <tr>
    <td style='border-right: 1px solid black;'>$c_{0}$</td>
    <td>$a_{10}$</td>
    <td>$a_{11}$</td>
    <td>0</td>
    <td>$\ddots$</td>
  </tr>
  <tr>
    <td style='border-right: 1px solid black;'>$\vdots$</td>
    <td>$\vdots$</td>
    <td>$\vdots$</td>
    <td>$\ddots$</td>
    <td>0</td>
  </tr>
  <tr>
    <td style='border-right: 1px solid black;'>$c_{s-1}$</td>
    <td>$a_{s-1,0}$</td>
    <td>$a_{s-1,1}$</td>
    <td>$\dots$</td>
    <td>$a_{s-1,s-1}$</td>
  </tr>
  <tr style='border-top: 1px solid black;'>
    <td style='border-right: 1px solid black;'>&nbsp;</td>
    <td>$b_{0}$</td>
    <td>$b_{1}$</td>
    <td>$\dots$</td>
    <td>$b_{s-1}$</td>
  </tr>
</table> 

For an SDIRK method we have $a∗=a_{ii}, i=0,\dots,s−1$.

We can use for example the 2 stage SDIRK (order 3) method

 <table style="width:50%; border-collapse:collapse;">
  <tr>
    <td style='border-right: 1px solid black;'>p</th>
    <td>p</td>
    <td>0</td>
  </tr>
  <tr>
    <td style='border-right: 1px solid black;'>1-p</td>
    <td>1-2p</td>
    <td>p</td>
  </tr>
  <tr style='border-top: 1px solid black;'>
    <td style='border-right: 1px solid black;'>&nbsp;</td>
    <td>1/2</td>
    <td>1/2</td>
  </tr>
</table> 
with $p=\frac{3−\sqrt{3}}{6}$.

In [20]:
class sdirk1: #order 1 (implicit Euler)
    stages = 1
    a = [[1]]
    b = [1]
    c = [1]
    astar = 1
    
class sdirk2: #order 3
    p = (3 - sqrt(3))/6
    stages = 2
    a = [[p, 0], 
         [1 - 2*p, p]]
    b = [1/2, 1/2]
    c = [p, 1 - p]
    astar = p
    
class sdirk5: #order 4
    stages = 5
    a = [[1/4, 0, 0, 0, 0], 
         [1/2, 1/4, 0, 0, 0], 
         [17/50,-1/25, 1/4, 0, 0],
         [371/1360, -137/2720, 15/544, 1/4,0],
         [25/24, -49/48, 125/16, -85/12, 1/4]]
    b = [25/24, -49/48, 125/16, -85/12, 1/4]
    c = [1/4, 3/4, 11/20, 1/2, 1]
    astar = 1/4    

butchertab = sdirk2()    
    
rhsi = gfu.vec.CreateVector()
Mu0 = gfu.vec.CreateVector()
ui = gfu.vec.CreateVector()
k = [gfu.vec.CreateVector() for i in range(butchertab.stages)]

We have to update the $M^∗$ matrix and reset initial data

In [21]:
time = 0.0
t.Set(0.0)
gfu.Set(uD,BND)
Draw(gfu,mesh,"u")
# visualization stuff
from ngsolve.internal import *
# visoptions.mminval = 0.0
# visoptions.mmaxval = 0.2
# visoptions.deformation = 0
# visoptions.autoscale = 0

mstar.AsVector().data = m.mat.AsVector() + butchertab.astar * dt * a.mat.AsVector()
invmstar = mstar.Inverse(freedofs=fes.FreeDofs())
invmass = m.mat.Inverse(freedofs=fes.FreeDofs())

In [22]:
tstep = 10 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
while t_intermediate < tstep - 0.5 * dt:
    Mu0.data = m.mat * gfu.vec
    for i in range(butchertab.stages):
        # add up the ks as prescribed in the butcher tableau
        rhsi.data = Mu0
        for j in range(0,i):
            rhsi.data += dt * butchertab.a[i][j] * k[j]
        
        # Solve for ui (with homogenization for the boundary data)
        t.Set(time+t_intermediate+butchertab.c[i]*dt)
        gfu.Set(uD,BND)
        ui.data = gfu.vec
        rhsi.data -= mstar * ui
        ui.data += invmstar * rhsi
        
        # compute k[i] from ui
        k[i].data = - a.mat * ui
        
    t_intermediate += dt
    t.Set(time+t_intermediate)
    
    # Adding up the ks:
    gfu.Set(uD,BND)
    Mu0.data -= m.mat * gfu.vec
    for i in range(0,butchertab.stages):
        Mu0.data += dt * butchertab.b[i] * k[i]
    gfu.vec.data += invmass * Mu0
        
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    
print("")
time+=t_intermediate    

 0.4130000000000003

 9.999999999999897675.58400000000019959.565000000000138


## Supplementary 4: VTK Output ("exporting the nice pictures")

- see also https://ngsolve.org/blog/ngsolve/2-vtk-output
- or py_tutorials/vtkout.py (ngsolve repository)

Outputting the nice pictures to vtk (to visualize with paraview):


In [20]:
vtk = VTKOutput(ma=mesh,coefs=[gfu],
                names=["sol"],
                filename="vtk_example1",
                subdivision=3)
vtk.Do()


You can also export vector fields:

In [21]:
vtk = VTKOutput(ma=mesh,coefs=[gfu,grad(gfu)],names=["sol","gradsol"],filename="vtk_example2",subdivision=3)
vtk.Do()

And time dependent data:

In [22]:
vtk = VTKOutput(ma=mesh,coefs=[gfu],names=["sol"],filename="vtk_example3",subdivision=3)
vtk.Do()

gfu.Set((1-y*y)*x)

time = 0
tstep = 1 # time that we want to step over within one block-run
t_intermediate=0 # time counter within one block-run
res = gfu.vec.CreateVector()
i = 0
while t_intermediate < tstep - 0.5 * dt:
    res.data = dt * f.vec - dt * a.mat * gfu.vec
    gfu.vec.data += invmstar * res
    t_intermediate += dt
    print("\r",time+t_intermediate,end="")
    Redraw(blocking=True)
    i += 1
    if (i%10 == 0):
        vtk.Do()
print("")

 1.0000000000000007760.1360000000000001
